In [1]:
import tensorflow as tf

v1 = tf.Variable(tf.constant(1.0, shape=[1], name="v1"))
v2 = tf.Variable(tf.constant(13.8, shape=[1], name="v2"))

result = v1 + v2

init_op = tf.global_variables_initializer()

saver = tf.train.Saver()

ckpt_path = "./demo/model/model.ckpt"

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, ckpt_path)
    
#with tf.Session() as sess2: ## 直接加载模型
#    saver.restore(sess2, ckpt_path)
#    print(sess2.run(result))
    
with tf.Session() as sess3:
    saver2 = tf.train.import_meta_graph(ckpt_path + ".meta")
    saver2.restore(sess3, ckpt_path)
    print(sess3.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from ./demo/model/model.ckpt
[14.8]


In [1]:
import tensorflow as tf

v1 = tf.Variable(tf.constant(1.0, shape=[1], name="other-v1"))
v2 = tf.Variable(tf.constant(13.8, shape=[1], name="other-v2"))

result = v1 + v2

init_op = tf.global_variables_initializer()

saver = tf.train.Saver({"v1": v1, "v2": v2})

ckpt_path = "./demo/model/model.ckpt"

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, ckpt_path)

with tf.Session() as sess2: ## 直接加载模型
    saver.restore(sess2, ckpt_path)
    print(sess2.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from ./demo/model/model.ckpt
[14.8]


In [1]:
# 先restart..
import tensorflow as tf

v = tf.Variable(0, dtype=tf.float32, name="v")

print("111")
for variables in tf.global_variables():
    print(variables.name)
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())

print("222")
for variables in tf.global_variables():
    print(variables.name)

saver = tf.train.Saver()

ckpt_path = "./demo/model/moving_average_model.ckpt"
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    saver.save(sess, ckpt_path)
    print(sess.run([v, ema.average(v)]))

111
v:0
222
v:0
v/ExponentialMovingAverage:0
[10.0, 0.099999905]


In [3]:
saver = tf.train.Saver({"v/ExponentialMovingAverage": v}) # 接着上面的那个一起跑，将v的滑动平均值重命名为v
ckpt_path = "./demo/model/moving_average_model.ckpt"
with tf.Session() as sess:
    saver.restore(sess, ckpt_path)
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from ./demo/model/moving_average_model.ckpt
0.099999905


In [4]:
print(ema.variables_to_restore())
saver = tf.train.Saver(ema.variables_to_restore()) # 接着上面的上面的那个一起跑
ckpt_path = "./demo/model/moving_average_model.ckpt"
with tf.Session() as sess:
    saver.restore(sess, ckpt_path)
    print(sess.run(v))

{'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>}
INFO:tensorflow:Restoring parameters from ./demo/model/moving_average_model.ckpt
0.099999905


# 将变量和取值以常量形式放在同一个文件中
默认tf.train.Saver会保存运行tf所需的全部信息，但在测试/离线预测时，并不需要这么多

In [5]:
import tensorflow as tf

from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2

init_op = tf.global_variables_initializer()

constant_ckpt_path = "./demo/model/constant_model.pb"

with tf.Session() as sess:
    sess.run(init_op)
    
    graph_def = tf.get_default_graph().as_graph_def()
    
    # "add"是需要保存的节点名字，注意，是节点名字，所以不要:0(:0表示第0个输出)
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ["add"])
    
    with tf.gfile.GFile(constant_ckpt_path, "wb") as f:
        f.write(output_graph_def.SerializeToString())


INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [7]:
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    with gfile.FastGFile(constant_ckpt_path, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    # 这里add:0是张量名称，不是计算节点
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    
    print(sess.run(result))

[array([3.], dtype=float32)]


# saver解析

In [8]:
import tensorflow as tf

v1 = tf.Variable(tf.constant(1.0, shape=[1], name="v1"))
v2 = tf.Variable(tf.constant(13.8, shape=[1], name="v2"))

result = v1 + v2


saver = tf.train.Saver()

ckpt_json_path = "./demo/model/model.ckpt.meta.json"

saver.export_meta_graph(ckpt_json_path, as_text=True)

meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "Assign"
      input_arg {
        name: "ref"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "value"
        type_attr: "T"
      }
      output_arg {
        name: "output_ref"
        type_attr: "T"
        